In [23]:
# Import relevant libraries
import os
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT
from SALib.sample import saltelli
import json
from ModularCirc import BatchRunner
from comparative_gsa.simulate_data import simulate_data
from comparative_gsa.calculate_output_features import calculate_output_features

In [24]:
param_path = '../inputs/parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas.json'
# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

n_samples = 10**7

In [25]:
# Set up the batch runner

## read in save parameters to the folder where simulations r saved.
with open(param_path, 'r') as f:
    params = json.load(f)

# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here?
br.setup_sampler(param_path)
br.sample(n_samples)

/Users/lbokeria/projects/health_gc/cvd_net_project/comparative-gsa/venv/lib/python3.12/site-packages/ModularCirc/_BatchRunner.py:59: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples  = self._sampler.random(nsamples)


In [26]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.v_ref,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,v_tot
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,...,225,25,150,10,0,420,25,280,800,5000
1,224.490389,0.267449,1391.318553,3.568193,9.100033,91.786620,6.305379,4.954608,0.372932,0.446568,...,225,25,150,10,0,420,25,280,800,5000
2,254.249478,0.327790,906.121586,2.608205,10.275423,104.035536,5.166425,4.290832,0.334445,0.542466,...,225,25,150,10,0,420,25,280,800,5000
3,271.680109,0.230533,1251.458715,3.003151,8.699872,129.094902,6.925526,3.321711,0.286747,0.613231,...,225,25,150,10,0,420,25,280,800,5000
4,285.041178,0.313907,1283.209757,3.239476,7.965886,122.133654,4.714354,4.508952,0.315288,0.459171,...,225,25,150,10,0,420,25,280,800,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,285.543928,0.281855,1207.640036,3.492069,11.246973,90.396597,6.565559,4.803102,0.314227,0.565995,...,225,25,150,10,0,420,25,280,800,5000
9999996,270.161525,0.374865,1309.716384,3.724678,10.478031,111.539389,5.776708,4.052576,0.287809,0.446126,...,225,25,150,10,0,420,25,280,800,5000
9999997,255.778018,0.277646,1094.224061,2.660298,8.762062,136.598538,6.314494,5.021158,0.333163,0.650360,...,225,25,150,10,0,420,25,280,800,5000
9999998,223.977730,0.336783,1170.394597,3.163255,7.862409,120.743761,5.156590,4.215243,0.373920,0.612470,...,225,25,150,10,0,420,25,280,800,5000


In [27]:
br._parameters_2_sample

{'ao.r': (195.0, 325.0),
 'ao.c': (0.22499999999999998, 0.375),
 'art.r': (843.75, 1406.25),
 'art.c': (2.25, 3.75),
 'ven.r': (6.75, 11.25),
 'ven.c': (84.975, 141.625),
 'av.r': (4.5, 7.5),
 'mv.r': (3.0749999999999997, 5.125),
 'la.E_pas': (0.22499999999999998, 0.375),
 'la.E_act': (0.41250000000000003, 0.6875),
 'la.k_pas': (0.00666, 0.03),
 'lv.E_pas': (0.75, 1.25),
 'lv.E_act': (2.25, 3.75),
 'lv.k_pas': (0.00666, 0.03)}

In [28]:
# Now, generate samples for sensitivity analysis by taking the keys of _parameters_2_sample and turning into a list
relevant_columns = list(br._parameters_2_sample.keys())
relevant_columns

['ao.r',
 'ao.c',
 'art.r',
 'art.c',
 'ven.r',
 'ven.c',
 'av.r',
 'mv.r',
 'la.E_pas',
 'la.E_act',
 'la.k_pas',
 'lv.E_pas',
 'lv.E_act',
 'lv.k_pas']

In [29]:
problem = {
    'num_vars': len(relevant_columns),
    'names': relevant_columns,
    'bounds' : br.samples[relevant_columns].describe().loc[['min', 'max']].T.values
}

In [30]:
param_values = saltelli.sample(problem, 4096*2, calc_second_order=True)
param_values.shape

/var/folders/rc/594kgpld1czfpr55m3_m1szw0000gr/T/ipykernel_70195/2667588959.py:1: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 4096*2, calc_second_order=True)


(245760, 14)

In [31]:
# How many rows are in param_values
param_values.shape[0]

# Truncate br.samples to be only the first n_samples rows
br._samples = br._samples.iloc[:param_values.shape[0]].copy()

In [32]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.v_ref,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,v_tot
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,...,225,25,150,10,0,420,25,280,800,5000
1,224.490389,0.267449,1391.318553,3.568193,9.100033,91.786620,6.305379,4.954608,0.372932,0.446568,...,225,25,150,10,0,420,25,280,800,5000
2,254.249478,0.327790,906.121586,2.608205,10.275423,104.035536,5.166425,4.290832,0.334445,0.542466,...,225,25,150,10,0,420,25,280,800,5000
3,271.680109,0.230533,1251.458715,3.003151,8.699872,129.094902,6.925526,3.321711,0.286747,0.613231,...,225,25,150,10,0,420,25,280,800,5000
4,285.041178,0.313907,1283.209757,3.239476,7.965886,122.133654,4.714354,4.508952,0.315288,0.459171,...,225,25,150,10,0,420,25,280,800,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245755,285.033139,0.232307,869.958782,3.598806,7.156085,123.430256,7.025206,3.661466,0.296717,0.604820,...,225,25,150,10,0,420,25,280,800,5000
245756,271.687933,0.310963,1112.647238,3.459350,7.819829,130.405363,4.638209,4.907856,0.305319,0.449724,...,225,25,150,10,0,420,25,280,800,5000
245757,254.241680,0.255857,1334.972963,2.928563,9.470156,105.345997,7.475790,3.955657,0.278163,0.653690,...,225,25,150,10,0,420,25,280,800,5000
245758,224.498434,0.349049,973.054563,3.269997,10.538172,89.597974,5.616133,3.207814,0.241430,0.558380,...,225,25,150,10,0,420,25,280,800,5000


In [33]:
n_samples = param_values.shape[0]

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

# Make this directory if it doesn't exist
os.makedirs(simulation_out_path, exist_ok=True)

In [34]:
# Now finish setting up the batch runner
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}

# Map the sample timings
br.map_sample_timings(
    ref_time=1000.,
    map=map_
    )

In [35]:
# Map the vessel volumes
br.map_vessel_volume()


# Save the samples to a CSV file
br.samples.to_csv(os.path.join(simulation_out_path,
                                f'input_samples_{n_samples}.csv'),
                                index=False)

# Set up the model with the parameters and time setup
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                time_setup=TEMPLATE_TIME_SETUP_DICT)

In [ ]:
simulations, bool_indices = simulate_data(
    batch_runner=br,
    simulation_out_path=simulation_out_path,
    n_jobs=8
)

Running batch simulation with 245760 samples...


  0%|          | 520/245760 [00:47<6:08:22, 11.10it/s]

In [ ]:
summary_df = calculate_output_features(
        simulations=simulations,
        simulation_out_path=simulation_out_path)